In [2]:
!pip install pydantic-ai python-dotenv --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4

In [3]:
import os
import nest_asyncio
from typing import Dict, List, Optional

from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()


In [28]:
#### UTILITIES ####
from pydantic import BaseModel, Field

# formats a response for better LLM processing
def to_markdown(data, indent=0):
    markdown = ""
    if isinstance(data, BaseModel):
        data = data.model_dump()
    if isinstance(data, dict):
        for key, value in data.items():
            markdown += f"{'#' * (indent + 2)} {key.upper()}\n"
            if isinstance(value, (dict, list, BaseModel)):
                markdown += to_markdown(value, indent + 1)
            else:
                markdown += f"{value}\n\n"
    elif isinstance(data, list):
        for item in data:
            if isinstance(item, (dict, list, BaseModel)):
                markdown += to_markdown(item, indent)
            else:
                markdown += f"- {item}\n"
        markdown += "\n"
    else:
        markdown += f"{data}\n\n"
    return markdown

# mock database
shipping_info_db: Dict[str, str] = {
    "#12345": "Shipped on 2024-12-01",
    "#67890": "Out for delivery",
}

def print_all_messages(response):
    for message in response.all_messages():
        print(message)

In [5]:
#### SETUP MODEL ####
from pydantic_ai.models.openai import OpenAIModel

if "OPENAI_API_KEY" not in os.environ:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

model = OpenAIModel("gpt-4o")

In [6]:
#### CREATE RESPONSE MODEL ####

class ResponseModel(BaseModel):
    response: str
    needs_escalation: bool
    follow_up_required: bool
    sentiment: str = Field(description="Customer sentiment analysis")

In [7]:
### CREATE DEPENDENCIES ####

class Order(BaseModel):
    order_id: str
    status: str
    items: list[str]

class CustomerDetails(BaseModel):
    customer_id: str
    name: str
    email: str
    orders: Optional[List[Order]] = None

In [29]:
#### CREATE AGENTS ####
from pydantic_ai import Agent, RunContext, Tool, ModelRetry

agent = Agent(
    model=model,
    result_type=ResponseModel,
    deps_type=CustomerDetails,
    retries=3,
    system_prompt=(
        "You are an intelligent customer support agent. "
        "Analyze queries carefully and provide structured responses. "
        "Use tools to look up relevant information."
        "Always great the customer and provide a helpful response."
    ),  # These are known when writing the code
    tools=[Tool(get_shipping_info, takes_ctx=True)],  # Add tool via kwarg
)

# Add dynamic system prompt based on dependencies
@agent.system_prompt
async def add_customer_name(ctx: RunContext[CustomerDetails]) -> str:
    return f"Customer details: {to_markdown(ctx.deps)}"

In [30]:
#### TOOLS ####

@agent.tool_plain()
def get_shipping_status(order_id: str) -> str:
    """Get the shipping status for a given order ID."""
    shipping_status = shipping_info_db.get(order_id)
    if shipping_status is None:
        raise ModelRetry(
            f"No shipping information found for order ID {order_id}. "
            "Make sure the order ID starts with a #: e.g, #624743 "
            "Self-correct this if needed and try again."
        )
    return shipping_info_db[order_id]

In [31]:
customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com",
)

response = await agent.run("What's the status of my last order 12345?", deps=customer)

print_all_messages(response)
# print(response.usage())

ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Use tools to look up relevant information.Always great the customer and provide a helpful response.', timestamp=datetime.datetime(2025, 3, 27, 12, 47, 23, 304012, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\nNone\n\n', timestamp=datetime.datetime(2025, 3, 27, 12, 47, 23, 304117, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content="What's the status of my last order 12345?", timestamp=datetime.datetime(2025, 3, 27, 12, 47, 23, 304123, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request')
ModelResponse(parts=[ToolCallPart(tool_name='get_shipping_status', args='{"order_id":"12345"}', tool_call_id='call_v56NPAOwJ

In [32]:
print(response.data.model_dump_json(indent=2))

{
  "response": "Hi John, it seems there is no shipping information available for order ID 12345. Please check to ensure the order ID is correct and formatted as \"#\" followed by the number. If you have any additional details about the order, or need further assistance, feel free to let me know!",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "neutral"
}


In [33]:
print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.data.follow_up_required}\n"
    f"Needs Escalation: {response.data.needs_escalation}"
)

Customer Details:
Name: John Doe
Email: john.doe@example.com

Response Details:
Hi John, it seems there is no shipping information available for order ID 12345. Please check to ensure the order ID is correct and formatted as "#" followed by the number. If you have any additional details about the order, or need further assistance, feel free to let me know!

Status:
Follow-up Required: False
Needs Escalation: False


In [37]:
customer = CustomerDetails(
    customer_id="1",
    name="John Doe",
    email="john.doe@example.com"
)

response = await agent.run("Oh ok, the shipping order id was #12345.", deps=customer)

print_all_messages(response)
# print(response.usage())

ModelRequest(parts=[SystemPromptPart(content='You are an intelligent customer support agent. Analyze queries carefully and provide structured responses. Use tools to look up relevant information.Always great the customer and provide a helpful response.', timestamp=datetime.datetime(2025, 3, 27, 12, 51, 22, 869745, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), SystemPromptPart(content='Customer details: ## CUSTOMER_ID\n1\n\n## NAME\nJohn Doe\n\n## EMAIL\njohn.doe@example.com\n\n## ORDERS\nNone\n\n', timestamp=datetime.datetime(2025, 3, 27, 12, 51, 22, 869876, tzinfo=datetime.timezone.utc), dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='Oh ok, the shipping order id was #12345.', timestamp=datetime.datetime(2025, 3, 27, 12, 51, 22, 869885, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request')
ModelResponse(parts=[ToolCallPart(tool_name='get_shipping_status', args='{"order_id":"12345"}', tool_call_id='call_3JOUrc0ML0

In [38]:
print(response.data.model_dump_json(indent=2))

{
  "response": "I'm sorry, but it seems there is no shipping information available for order ID #12345 at the moment. Please double-check the order ID for any typographical errors, or let me know if there's anything else I can assist you with!",
  "needs_escalation": false,
  "follow_up_required": false,
  "sentiment": "neutral"
}


In [39]:
print(
    "Customer Details:\n"
    f"Name: {customer.name}\n"
    f"Email: {customer.email}\n\n"
    "Response Details:\n"
    f"{response.data.response}\n\n"
    "Status:\n"
    f"Follow-up Required: {response.data.follow_up_required}\n"
    f"Needs Escalation: {response.data.needs_escalation}"
)

Customer Details:
Name: John Doe
Email: john.doe@example.com

Response Details:
I'm sorry, but it seems there is no shipping information available for order ID #12345 at the moment. Please double-check the order ID for any typographical errors, or let me know if there's anything else I can assist you with!

Status:
Follow-up Required: False
Needs Escalation: False
